In [1]:
import numpy as np
import ufl
from mpi4py import MPI
from petsc4py import PETSc
from dolfinx import plot, io, fem, mesh, default_scalar_type
import dolfinx_mpc.utils
import dolfinx_mpc as dmpc
from dolfinx.io import XDMFFile, gmshio

import meshio
import gmsh
import math
import sys
import pyvista

gdim = 2

Just change the name from "fiber_mesh" in the line below to whatever your next mesh is called.

In [2]:
domain, cell_markers, facet_markers = gmshio.read_from_msh("RVE/meshes/xdmf_meshes/mesh1.msh", MPI.COMM_WORLD, gdim=gdim)
#domain,cells,facets
pyvista.start_xvfb()
pyvista.set_jupyter_backend('html')

# Create plotter and pyvista grid
p = pyvista.Plotter()
topology, cell_types, geometry = plot.vtk_mesh(domain)
grid = pyvista.UnstructuredGrid(topology, cell_types, geometry)

actor_0 = p.add_mesh(grid, show_edges=True)#style="wireframe", color="k")

p.show_axes()
if not pyvista.OFF_SCREEN:
   p.show()
else:
   figure_as_array = p.screenshot("mesh.png")

Info    : Increasing process stack size (8176 kB < 16 MB)
Info    : Reading 'RVE/xdmf_meshes/mesh1.msh'...
Info    : 14 entities
Info    : 458 nodes
Info    : 852 elements
Info    : Done reading 'RVE/xdmf_meshes/mesh1.msh'


ImportError: Please install `trame` and `ipywidgets` to use this feature.

In [ ]:
cells_0 = cell_markers.find(1)
cells_1 = cell_markers.find(2)

In [ ]:
pyvista.start_xvfb()
pyvista.set_jupyter_backend('html')
# Filter out ghosted cells
num_cells_local = domain.topology.index_map(domain.topology.dim).size_local
marker = np.zeros(num_cells_local, dtype=np.int32)
cells_0 = cells_0[cells_0<num_cells_local]
cells_1 = cells_1[cells_1<num_cells_local]
marker[cells_0] = 1
marker[cells_1] = 2
topology, cell_types, x = plot.vtk_mesh(domain, domain.topology.dim, np.arange(num_cells_local, dtype=np.int32))

p = pyvista.Plotter(window_size=[800, 800])
grid = pyvista.UnstructuredGrid(topology, cell_types, x)
grid.cell_data["Marker"] = marker
grid.set_active_scalars("Marker")
p.add_mesh(grid, show_edges=True)
if pyvista.OFF_SCREEN:
    figure = p.screenshot("subdomains_structured.png")
p.show()

In [ ]:
def create_piecewise_constant_field(domain, cell_markers, property_dict, name=None):
    """Create a piecewise constant field with different values per subdomain.

    Parameters
    ----------
    domain : Mesh
        `dolfinx` mesh object
    cell_markers : MeshTag
        cell marker MeshTag
    property_dict : dict
        A dictionary mapping region tags to physical values {tag: value}

    Returns
    -------
    A DG-0 function
    """
    V0 = fem.functionspace(domain, ("DG", 0))
    k = fem.Function(V0, name=name)
    for tag, value in property_dict.items():
        cells = cell_markers.find(tag)
        k.x.array[cells] = np.full_like(cells, value, dtype=np.float64)
    return k
    
E_mat = 3.45e9 #Just change these values to change the materials of the matrix
nu_mat = 0.35
E_fiber = 1.8e9 #Just change these values to change the materials of the fiber
nu_fiber = 0.43

E = create_piecewise_constant_field(
    domain, cell_markers, {1: E_mat, 2: E_fiber}, name="YoungModulus"
)
nu = create_piecewise_constant_field(
    domain, cell_markers, {1: nu_mat, 2: nu_fiber}, name="PoissonRatio"
)

lmbda = E * nu / (1 + nu) / (1 - 2 * nu)
mu = E / 2 / (1 + nu)



In [ ]:
xface = np.max(domain.geometry.x[:,0])
xface_neg = np.min(domain.geometry.x[:,0])
yface = np.max(domain.geometry.x[:,1])
yface_neg = np.min(domain.geometry.x[:,1])
# zface = np.max(domain.geometry.x[:,2])
# zface_neg = np.min(domain.geometry.x[:,2])
dx = xface-xface_neg
dy = yface-yface_neg
# dz = zface-zface_neg
#v_total = dx*dy*dz #volume within boundary of RVE
v_total = dx*dy
vol = fem.assemble_scalar(fem.form(1 * ufl.dx(domain=domain)))
boundaries = [(2, lambda x: np.isclose(x[0],xface)),
              (3, lambda x: np.isclose(x[1],yface)),
              (0, lambda x: np.isclose(x[0],xface_neg)),
              (1, lambda x: np.isclose(x[1],yface_neg))]

facet_indices, facet_markers = [], []
tdim = domain.topology.dim
fdim = tdim - 1
domain.topology.create_connectivity(fdim, tdim)
# fdim = domain.topology.dim - 1
for (marker, locator) in boundaries:
    facets = mesh.locate_entities(domain, fdim, locator)
    facet_indices.append(facets)
    facet_markers.append(np.full_like(facets, marker))
facet_indices = np.hstack(facet_indices).astype(np.int32)
facet_markers = np.hstack(facet_markers).astype(np.int32)
sorted_facets = np.argsort(facet_indices)
facet_tag = mesh.meshtags(domain, fdim, facet_indices[sorted_facets], facet_markers[sorted_facets])  
ds = ufl.Measure("ds", domain=domain, subdomain_data=facet_tag)
#DX = ufl.Measure("dx", domain=domain, subdomain_data=facet_tag)
n = ufl.FacetNormal(domain)

In [ ]:
corners = np.array([[xface_neg, yface_neg], [xface, yface_neg], [xface, yface], [xface_neg, yface]])
a1 = corners[1, :] - corners[0, :]  # first vector generating periodicity
a2 = corners[3, :] - corners[0, :]  # second vector generating periodicity

Eps = fem.Constant(domain, np.zeros((2, 2)))
Eps_ = fem.Constant(domain, np.zeros((2, 2)))
y = ufl.SpatialCoordinate(domain)


def epsilon(v):
    return ufl.sym(ufl.grad(v))


def sigma(v):
    eps = Eps + epsilon(v)
    return lmbda * ufl.tr(eps) * ufl.Identity(gdim) + 2 * mu * eps


V = fem.functionspace(domain, ("P", 2, (gdim,)))
du = ufl.TrialFunction(V)
u_ = ufl.TestFunction(V)
a_form, L_form = ufl.system(ufl.inner(sigma(du), epsilon(u_)) * ufl.dx)
# -

# ### Periodic boundary conditions enforcement using `dolfinx_mpc`
#
# We must now define the periodic boundary conditions for the fluctuation field. For that, we make use of `dolfinx_mpc` providing a `MultiPointConstraint` object which can account for periodicity conditions. Note that periodic conditions do not fix rigid body translations. To remove them we choose here, for simplicity, to fix the displacement of the single point of coordinate `(0, 0)`. An alternative can consist of introducing constant Lagrange multipliers as discussed in the legacy demo. This solution requires however to use `Real` elements which require special care and are currently available in the `scifem` package https://github.com/scientificcomputing/scifem.

point_dof = fem.locate_dofs_geometrical(
    V, lambda x: np.isclose(x[0], xface_neg) & np.isclose(x[1], yface_neg)
)
bcs = [fem.dirichletbc(np.zeros((gdim,)), point_dof, V)]


# We first instantiate the `MultiPointConstraint` object `mpc` defined with respect to our function space `V`. The function `create_periodic_constraint_topological` enables to link dofs on corresponding surfaces. We must first define a mapping between points on corresponding surfaces. For instance, we apply the first condition to the right surface, tagged `2`. The `periodic_relation_left_right` transforms input coordinates on the right surface to points on the left surface as follows: $(x, y) \mapsto (x-L_x, y)$. More generally, the mapping is $(x, y) \mapsto (x-a_{1x}, y-a_{1y})$ for the first periodicity-generating base vector $\ba_1$. We do the same for the top surface, tagged `3`, which is mapped to th
# e bottom one using the second periodicity-generating base vector $\ba_2.$

# +
def periodic_relation_left_right(x):
    out_x = np.zeros(x.shape)
    out_x[0] = x[0] - a1[0]
    out_x[1] = x[1] - a1[1]
    out_x[2] = x[2]
    return out_x


def periodic_relation_bottom_top(x):
    out_x = np.zeros(x.shape)
    out_x[0] = x[0] - a2[0]
    out_x[1] = x[1] - a2[1]
    out_x[2] = x[2]
    return out_x


mpc = dolfinx_mpc.MultiPointConstraint(V)
mpc.create_periodic_constraint_topological(
    V, facet_tag, 2, periodic_relation_left_right, bcs
)
mpc.create_periodic_constraint_topological(
    V, facet_tag, 3, periodic_relation_bottom_top, bcs
)
mpc.finalize()
# -

# ### Problem resolution
#
# We are now in position of defining the problem instance. Note that we must use here the `LinearProblem` provided by `dolfinx_mpc` and not that of `dolfinx.fem.petsc`. The former expands upon the latter by including the `MultiPointConstraint` object in its arguments. Note also that the solution function field $\bv$, and the related total displacement $\bu$, must be built from the reduced function space object provided by `mpc.function_space` and not `V` in order to properly account for the imposed kinematic constraint.

u = fem.Function(mpc.function_space, name="Displacement")
v = fem.Function(mpc.function_space, name="Periodic_fluctuation")
problem = dmpc.LinearProblem(
    a_form,
    L_form,
    mpc,
    bcs=bcs,
    u=v,
    petsc_options={"ksp_type": "preonly", "pc_type": "lu"},
)

# To solve the homogenization problem, we define a list of elementary load cases representing macroscopic uniaxial tension in both $x$ and $y$ directions and uniform macroscopic shear in the $xy$ direction. For each of such elementary load case, we update the value of the macroscopic strain constant $\bE$ and solve the corresponding problem. The microscopic total displacement $\bu(\by)$ is then built from the periodic fluctuation. For each load case, we compute the different components of the macroscopic average stress $\boldsymbol{\Sigma}$. Since we considered unit load cases, the latter directly provide the components of the macroscopic effective stiffness tensor $\mathbb{C}^\text{hom}$.

# + tags=["hide-input"]
import pyvista
from dolfinx import plot

pyvista.set_jupyter_backend("static")


def plot_warped(u, scale=1.0, plot_mesh=False, user_callback=None, title=None):
    Vu = u.function_space
    u_topology, u_cell_types, u_geometry = plot.vtk_mesh(Vu)
    u_grid = pyvista.UnstructuredGrid(u_topology, u_cell_types, u_geometry)
    u_3D = np.zeros((u_geometry.shape[0], 3))
    u_3D[:, :2] = u.x.array.reshape(-1, 2)
    u_grid.point_data[u.name] = u_3D
    u_grid.set_active_vectors(u.name)
    warped = u_grid.warp_by_vector(u.name, factor=scale)

    plotter = pyvista.Plotter()
    plotter.window_size = (800, 300)
    plotter.add_mesh(warped)
    if plot_mesh:
        edges = warped.extract_all_edges()
        plotter.add_mesh(edges, color="k", line_width=1, opacity=0.5)
    plotter.view_xy()
    if title:
        plotter.add_text(title, font_size=14)
    plotter.show()


# +
elementary_load = [
    np.array([[1.0, 0.0], [0.0, 0.0]]),
    np.array([[0.0, 0.0], [0.0, 1.0]]),
    np.array([[0.0, 0.5], [0.5, 0.0]]),
]
load_labels = ["Exx", "Eyy", "Exy"]
dim_load = len(elementary_load)

C_hom = np.zeros((dim_load, dim_load))
for nload in range(dim_load):
    Eps.value = elementary_load[nload]
    u.interpolate(
        fem.Expression(
            ufl.dot(Eps, y), mpc.function_space.element.interpolation_points()
        )
    )

    problem.solve()
    u.x.array[:] += v.x.array[:]

    plot_warped(u, scale=0.5, plot_mesh=True, title=load_labels[nload])

    for nload_ in range(dim_load):
        Eps_.value = elementary_load[nload_]

        C_hom[nload, nload_] = (
            fem.assemble_scalar(fem.form(ufl.inner(sigma(v), Eps_) * ufl.dx)) / vol
        )
# -


In [ ]:
C_hom